In [1]:
# CN solver with logic for American and European options

import numpy as np
from scipy import linalg

def CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = False):
    
    """alpha and beta are assumed to be functions of x and t
    phi1, phi2 are functions of xmin, t and xmax, t
    """

    dx = (xmax - xmin)/n
    x = np.linspace(xmin, xmax, n+1)
    
    # define the function alpha
    # for transformed BS equation
    dt = T/m
    
    # define values of x_i
    x = np.linspace(xmin, xmax, n+1)
    
    # define matrix U with dimension (n+1) by (m+1)
    U = np.zeros(shape = (n+1, m+1))
    
    # fill in the final condition
    U[:, m] = u0(x)
    
    # compute all other values
    i = np.arange(1, n)
    t = np.linspace(0, T, m+1)
    
    # define matrix M
    M = np.zeros(shape = (n+1, n+1))
    M[0, 0] = 1
    M[n, n] = 1
    
    # define vector F
    F = np.zeros(n+1)
    
    for k in range (m-1, -1, -1):
               
        aik = (-dt/(2*dx**2))*(alpha(x[i], t[k]) - beta(x[i], t[k])/2*dx)
        bik = 1 + (dt/(dx**2))*alpha(x[i], t[k]) + r*dt/2
        cik = (-dt/(2*dx**2))*(alpha(x[i], t[k]) + beta(x[i], t[k])/2*dx)
        dik = (dt/(2*dx**2))*(alpha(x[i], t[k+1]) - beta(x[i], t[k+1])/2*dx)
        eik = 1 - (dt/(dx**2))*alpha(x[i], t[k+1]) - r*dt/2
        fik = (dt/(2*dx**2))*(alpha(x[i], t[k+1]) + beta(x[i], t[k+1])/2*dx)
        
        M[i, i-1] = aik
        M[i, i] = bik
        M[i, i+1] = cik
    
        # boundary conditions
        F[0] = phi1(xmin, t[k])
        F[n] = phi2(xmax, t[k])
        F[i] = dik*U[i-1, k+1] + eik*U[i, k+1] + fik*U[i+1, k+1]
        
        # solve the system
        U[:, k] = linalg.solve(M, F)
        
        if American:
            U[:, k] = np.maximum(U[:, k], U[:, m])
        
    return U[:, 0] # the first column of U, option prices for t = 0

In [2]:
# parameters given in the problem

def sigma(s, t):
    return .3 + .3/(1 + .1*np.abs(s - 5*t - 55))

r = .02
D = .04
S0 = 70
T = .5
total_error = .01

rho = 2
xmin = np.log(S0/rho)
xmax = np.log(S0*rho)

def p(s):
    return np.maximum(np.abs(s - 65) - 15, 0)

def alpha(x, t):
    return sigma(np.exp(x), t)**2/2
    
def beta(x, t):
    return r - D - alpha(x, t)

def u0(x):
    return p(np.exp(x))

def phi1(xmin,t):
    return p(np.exp(xmin))

def phi2(xmax,t):
    return p(np.exp(xmax))

n0 = 10
m0 = 5
z = 1

In [3]:
# testing whether my CN solver can compute the price of an option correctly

n = z*n0
m = m0
American_option = CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = True)[n//2]
European_option = CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = False)[n//2]
print(American_option, European_option)

5.738042721608455 5.722526947799763


In [4]:
def Runge(m0, n0, mfactor, nfactor, q, value, error):
    """value is a function of m, n, should return answer (a number)
    if we multipy m by mfactor, n by nfactor, the error is reduced by q
    error is how accurate we want the answer to be
    """
    m = m0
    n = n0
    answer1 = value(m, n)
    estimate = error + 1
    while estimate > error:
        m = m*mfactor
        n = n*nfactor
        answer2 = value(m, n)
        estimate = np.abs(answer2 - answer1)/(q - 1)
        answer1 = answer2
        print("runge", m, n, rho, estimate)
    return answer2

In [5]:
def price_CN(m,n):
    return CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = True)[n//2]

In [6]:
answer_rho1 = Runge(10, 10, 4, 2, 2, price_CN, total_error/2)
estimate_rho = total_error
z = 1
while (estimate_rho > total_error/2):
    z = z + 1
    rho = rho*2
    xmin = np.log(S0/rho)
    xmax = np.log(S0*rho)
    answer_rho2 = Runge(10, 10*z, 4, 2, 2, price_CN, total_error/2)
    estimate_rho = np.abs(answer_rho1 - answer_rho2)
    answer_rho1 = answer_rho2
    price_American_option = answer_rho2
print("the price of the American option is",price_American_option)

# runge 40 20 2 0.17979624293731167
# runge 160 40 2 0.038725334838956904
# runge 640 80 2 0.007329760289719545
# runge 2560 160 2 0.0005240392022791696
# runge 40 40 4 0.1772018079582569
# runge 160 80 4 0.038175540528351704
# runge 640 160 4 0.007182949871583411
# runge 2560 320 4 0.00048388207738447875
# the price of the American option is 5.972539932740992

runge 40 20 2 0.17979624293731167
runge 160 40 2 0.038725334838956904
runge 640 80 2 0.007329760289719545
runge 2560 160 2 0.0005240392022791696
runge 40 40 4 0.1772018079582569
runge 160 80 4 0.038175540528351704
runge 640 160 4 0.007182949871583411
runge 2560 320 4 0.00048388207738447875
the price of the American option is 5.972539932740992


In [7]:
def derivative(m, n, American, i_price):
    prices = CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = True)
    dx = (xmax - xmin)/n
    return (prices[i_price + 1] - prices[i_price - 1])/(2*dx)/S0

derivative(m, n, True, n//2)

# 0.2412279559040229
# American

0.2412279559040229

In [8]:
American = True
def value(m, n):
    # when using an explicit method, just ignore the value of m
    return derivative(m, n, American, n//2)
answer_rho1 = Runge(10, 10, 4, 2, 2, value, total_error/2)
estimate_rho = total_error
z = 1
while (estimate_rho > total_error/2):
    z = z + 1
    rho = rho*2
    # answer_rho2 = Runge(10, 10*z, 2, 2, 4, value, total_error/2)
    answer_rho2 = Runge(10, 10*z, 4, 2, 2, value, total_error/2)
    estimate_rho = np.abs(answer_rho1 - answer_rho2)
    answer_rho1 = answer_rho2

delta_American_option = answer_rho2

print("the delta of the American option is",delta_American_option)

# runge 40 20 4 0.05229830079113726
# runge 160 40 4 0.007165703948753321
# runge 640 80 4 0.0018496236500237517
# runge 40 40 8 0.007060875646118597
# runge 160 80 8 0.001808642000808186
# the delta of the American option is 0.1802408163464242

runge 40 20 4 0.05229830079113726
runge 160 40 4 0.007165703948753321
runge 640 80 4 0.0018496236500237517
runge 40 40 8 0.007060875646118597
runge 160 80 8 0.001808642000808186
the delta of the American option is 0.1802408163464242


In [9]:
# For the European case:

rho = 2
xmin = np.log(S0/rho)
xmax = np.log(S0*rho)
n0 = 10
m0 = 5
z = 1
n = n0
m = m0

def price_CN(m,n):
    return CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = False)[n//2]

def derivative(m, n, American, i_price):
    prices = CN(m, n, xmin, xmax, r, T, alpha, beta, u0, phi1, phi2, American = False)
    dx = (xmax - xmin)/n
    return (prices[i_price + 1] - prices[i_price - 1])/(2*dx)/S0

derivative(m, n, False, n//2)

answer_rho1 = Runge(10, 10, 4, 2, 2, price_CN, total_error/2)
estimate_rho = total_error
z = 1
while (estimate_rho > total_error/2):
    z = z + 1
    rho = rho*2
    xmin = np.log(S0/rho)
    xmax = np.log(S0*rho)
    answer_rho2 = Runge(10, 10*z, 4, 2, 2, price_CN, total_error/2)
    estimate_rho = np.abs(answer_rho1 - answer_rho2)
    answer_rho1 = answer_rho2
    price_European_option = answer_rho2
print("the price of the European option is",price_European_option)

# runge 40 20 2 0.17169573777524327
# runge 160 40 2 0.035587023942511564
# runge 640 80 2 0.006157693764758498
# runge 2560 160 2 9.995110571114907e-05
# runge 40 40 4 0.170593797272228
# runge 160 80 4 0.03543025595205851
# runge 640 160 4 0.006108807130388527
# runge 2560 320 4 8.424605760470882e-05
# the price of the European option is 5.9390908389878785

runge 40 20 2 0.17169573777524327
runge 160 40 2 0.035587023942511564
runge 640 80 2 0.006157693764758498
runge 2560 160 2 9.995110571114907e-05
runge 40 40 4 0.170593797272228
runge 160 80 4 0.03543025595205851
runge 640 160 4 0.006108807130388527
runge 2560 320 4 8.424605760470882e-05
the price of the European option is 5.9390908389878785


In [10]:
American = False
def value(m, n):
    #in the case of using an explicit method, we just ignore the value of m
    return derivative(m, n, American, n//2)
answer_rho1 = Runge(10, 10, 4, 2, 2, value, total_error/2)
estimate_rho = total_error
z = 1
while (estimate_rho > total_error/2):
    z = z + 1
    rho = rho*2
    answer_rho2 = Runge(10, 10*z, 4, 2, 2, value, total_error/2)
    estimate_rho = np.abs(answer_rho1 - answer_rho2)
    answer_rho1 = answer_rho2

delta_European_option = answer_rho2

print("the delta of the European option is",delta_European_option)

# runge 40 20 4 0.05216656624640853
# runge 160 40 4 0.007210369033354341
# runge 640 80 4 0.0019301410971800936
# runge 40 40 8 0.007323710542540995
# runge 160 80 8 0.0019347345168801433
# the delta of the European option is 0.17700415174287917

runge 40 20 4 0.05216656624640853
runge 160 40 4 0.007210369033354341
runge 640 80 4 0.0019301410971800936
runge 40 40 8 0.007323710542540995
runge 160 80 8 0.0019347345168801433
the delta of the European option is 0.17700415174287917


In [11]:
print(price_American_option, price_European_option, delta_American_option, delta_European_option)

# 5.972539932740992 5.9390908389878785 0.1802408163464242 0.17700415174287917

5.972539932740992 5.9390908389878785 0.1802408163464242 0.17700415174287917


In [12]:
print("The price of the American option is",price_American_option)
print("The price of the European option is",price_European_option)
print("The delta of the American option is", delta_American_option)
print("The delta of the European option is",delta_European_option)

# 5.972539932740992 5.9390908389878785 0.1802408163464242 0.17700415174287917

The price of the American option is 5.972539932740992
The price of the European option is 5.9390908389878785
The delta of the American option is 0.1802408163464242
The delta of the European option is 0.17700415174287917
